In [4]:
import pandas as pd
import numpy as np


### Command for data Generation sacct -n --delimiter="$" -a -S 2020-02-20T00:00:00 -E 2020-02-21T23:59:59 --format "User,JobID,ReqCPUS,ReqMem,ReqNodes,Start,End,CPUTimeRAW" | tr -s ' ' | awk '{$1=$1};1'

inputFile = ""
outputFile = ""

### Load in Data 
columns = ["user","jobid","reqcpu","reqmem","reqnode","start","end","cputime"]
df = pd.read_csv(inputFile,delimiter=" ")
df.columns = columns
### Generate List of Users
usersList = []
for i in df["user"].unique():
    if i.isalpha():
        usersList.append(i)
## Shrink DataFrame to contain only single Jobs with 1 core and 1 node
df = df[(df['user'].isin(usersList))]
df = df[df["end"]!="Unknown"]
df = df[df["start"]!="Unknown"]
df['start'] = pd.to_datetime(df['start'], format='%Y-%m-%dT%H:%M:%S')
df['end'] = pd.to_datetime(df['end'], format='%Y-%m-%dT%H:%M:%S')
df['jobTime'] = (df['end'] - df['start'])/ np.timedelta64(1, 's')
### OSG Jobs are single node and core and are a day or less
osgdf = df[(df['reqcpu'] == 1) & (df['reqnode'] == 1) & (df['jobTime'] <= 86400)  ]
print("Data Loaded")
osgData = {}
details = 0
for user in usersList:
    totalJobs = len(df[df['user'] == user])
    for r,i in osgdf[osgdf['user'] == user].iterrows():   
        osgIdeal = 0
        osgLimit = 0
        memreq = 0
        timeToFinish = i["jobTime"]
        if "M" in i["reqmem"]:
            memreq = float(i["reqmem"].split("M")[0])
        if "G" in i["reqmem"]:
            memreq = float(i["reqmem"].split("G")[0]) * 1024
        if timeToFinish <= 36000 and memreq <= 4096:
            osgIdeal = 1
        if timeToFinish > 36000 and memreq > 4096 and memreq <= 10240:
            osgLimit = 1
    
        if user in osgData.keys(): 
            osgData[user] = {
                "jobtotal": totalJobs,
                "osgIdeal": osgData[user]["osgIdeal"] + osgIdeal,
                "osgLimit": osgData[user]["osgLimit"] + osgLimit
            }
        else:
            osgData[user] = {
                "jobtotal": totalJobs,
                "osgIdeal": osgIdeal,
                "osgLimit": osgLimit
            }
print("Writing Data")
with open(outputFile,'a') as f:
    f.write("user,osgIdeal,osgLimit,total,percIdeal,percLimit\n")
    for k,v in osgData.items():
        f.write("%s,%s,%s,%s,%s,%s\n" % (k,str(v["osgIdeal"]),str(v["osgLimit"]),str(v["jobtotal"]),str(v["osgIdeal"]/v["jobtotal"]),str(v["osgLimit"]/v["jobtotal"])))


/util/opt/anaconda/4.3.14/envs/jupyterhub-root/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Time To Load Data: 8.420723 Seconds
Time To Gen Users + Load data: 8.745385 Seconds
Time To Shrink DF + Gen Users + Load data: 20.626690 Seconds
Data Loaded
0 82
1 82
2 82
3 82
4 82
5 82
6 82
7 82
8 82
9 82
10 82
11 82
12 82
13 82
14 82
15 82
16 82
17 82
18 82
19 82
20 82
21 82
22 82
23 82
24 82
25 82
26 82
27 82
28 82
29 82
30 82
31 82
32 82
33 82
34 82
35 82
36 82
37 82
38 82
39 82
40 82
41 82
42 82
43 82
44 82
45 82
46 82
47 82
48 82
49 82
50 82
51 82
52 82
53 82
54 82
55 82
56 82
57 82
58 82
59 82
60 82
61 82
62 82
63 82
64 82
65 82
66 82
67 82
68 82
69 82
70 82
71 82
72 82
73 82
74 82
75 82
76 82
77 82
78 82
79 82
80 82
81 82
82 82
Writing Data
